In [ ]:
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image

import pandas as pd
import numpy as np

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl

from resources.plotly_theme import nfl_template
from resources.heat_map import heat_map
from resources.get_nfl_data import get_pbp_data, get_team_data

from notebooks.resources.team_stats import get_team_offense_data

pio.templates['nfl_template'] = nfl_template

In [ ]:
''' Parameters '''

WEEK = 8
AWAY_TEAM = 'WAS'
HOME_TEAM = 'KC'


''' Import Data '''

# Import
team_data = get_team_data()
pbp_data = get_pbp_data(years=[2025])

player_info = nfl.import_players()

game_data = pbp_data.loc[(pbp_data['week'] == WEEK) & 
                         (pbp_data['home_team'] == HOME_TEAM), :]

print(game_data.shape)

### "pass" and "rush"
 - *Designed* pass or rush, regardless of outcome
   - QB scrambles, sacks, penalties, blown dead, etc.

### dropback
 - Non-penalty designed pass plays
 - INCLUDES eventual qb scrambles

### Sacks
 - Count as dropback, pass_attempt, pass, and sack

### Rushing
 - per "Designed Rush"
    - EPA per Designed Rush
    - Success Rate on Designed Rushes
    - where rush == 1

### Passing
 - per "Dropback"
    - EPA per Dropback
    - Success Rate on Dropbacks
    - where pass == 1
    - includes QB scrambles, which are rush attempts


In [ ]:
''' Game '''

team_offense = game_data.loc[game_data['Offensive Snap'], :].groupby(['posteam']).aggregate(
    Plays=('posteam', 'size'),
    Yards=('yards_gained', 'sum'),
    RushAttempts=('rush_attempt', 'sum'),
    DesignedRushes=('rush', 'sum'),
    QBScrambles=('qb_scramble', 'sum'),
    RushYards=('rushing_yards', 'sum'),#, lambda x: x[game_data['rush'] == 1].sum()),
    DesignedPasses=('pass', 'sum'),
    Dropbacks=('qb_dropback', 'sum'),
    PassCompletions=('complete_pass', 'sum'),
    PassAttempts=('pass_attempt', 'sum'),
    PassYards=('passing_yards', 'sum'),# lambda x: x[game_data['pass'] == 1].sum()),
    PassYardsTot=('yards_gained', lambda x: x[game_data['pass'] == 1].sum()),
    Sacks=('sack', 'sum'),
    SackYards=('yards_gained', lambda x: x[game_data['sack'] == 1].sum()),
    EPA=('epa', 'sum'),
    RushEPA=('epa', lambda x: x[game_data['rush'] == 1].sum()),
    PassEPA=('epa', lambda x: x[game_data['pass'] == 1].sum()),
    Successes=('success', 'sum'),
    RushSuccesses=('success', lambda x: x[game_data['rush'] == 1].sum()),
    PassSuccesses=('success', lambda x: x[game_data['pass'] == 1].sum()),
)
team_offense['PassAttempts'] = team_offense['PassAttempts'] - team_offense['Sacks']
team_offense['EPA / Play'] = round(team_offense['EPA'] / team_offense['Plays'], 2)
team_offense['Rush EPA / Play'] = round(team_offense['RushEPA'] / team_offense['DesignedRushes'], 2)
team_offense['Pass EPA / Play'] = round(team_offense['PassEPA'] / team_offense['DesignedPasses'], 2)
team_offense['Success Rate'] = round(team_offense['Successes'] / team_offense['Plays'], 2)
team_offense['Rush Success Rate'] = round(team_offense['RushSuccesses'] / team_offense['DesignedRushes'], 2)
team_offense['Pass Success Rate'] = round(team_offense['PassSuccesses'] / team_offense['DesignedPasses'], 2)

print(team_offense.to_string())

In [ ]:
''' Team Standard '''

ROUND = 3

team_offense = pbp_data.loc[(~pbp_data['Is Special Teams Play']), :].groupby(['posteam']).aggregate(
    Games=('game_id', 'nunique'),
    Plays=('posteam', lambda x: x[(pbp_data['rush_attempt'] == 1) | (pbp_data['pass_attempt'] == 1)].shape[0]),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),

    RushAttempts=('rush_attempt', 'sum'),
    RushYards=('rushing_yards', 'sum'),#, lambda x: x[pbp_data['rush'] == 1].sum()),
    RushTDs=('rush_touchdown', 'sum'),
    Rush1Ds=('first_down_rush', 'sum'),
    DesignedRushPlays=('rush', 'sum'),
    DesignedRushAttempts=('rush_attempt', lambda x: x[pbp_data['rush'] == 1].sum()),
    DesignedRushYards=('rushing_yards', lambda x: x[(pbp_data['rush'] == 1)].sum()),
    QBScrambles=('qb_scramble', lambda x: x[pbp_data['rush_attempt'] == 1].sum()),
    ScrambleYards=('rushing_yards', lambda x: x[(pbp_data['qb_scramble'] == 1) & (pbp_data['rush_attempt'] == 1)].sum()),

    DesignedPassPlays=('pass', 'sum'),
    Dropbacks=('qb_dropback', 'sum'),
    PassCompletions=('complete_pass', 'sum'),
    PassAttempts=('pass_attempt', 'sum'),
    PassYards=('passing_yards', 'sum'),# lambda x: x[pbp_data['pass'] == 1].sum()),
    PassTDs=('pass_touchdown', 'sum'),
    Pass1Ds=('first_down_pass', 'sum'),

    Sacks=('sack', 'sum'),
    SackYards=('yards_gained', lambda x: x[pbp_data['sack'] == 1].sum()),
    INTs=('interception', 'sum'),

    Fumbles=('fumble_lost', 'sum'),

    Penalties=('penalty', lambda x: x[pbp_data['penalty_team'] == pbp_data['posteam']].sum()),
    PenaltyYards=('penalty_yards', lambda x: x[pbp_data['penalty_team'] == pbp_data['posteam']].sum()),
    Penalty1Ds=('first_down_penalty', 'sum'),# lambda x: x[pbp_data['penalty_team'] == pbp_data['defteam']].sum()),

    Drives=('drive_play_id_started', 'nunique'),
)

# Adjustments
team_offense['PassYards'] = team_offense['PassYards'] + team_offense['SackYards']
team_offense['PassAttempts'] = team_offense['PassAttempts'] - team_offense['Sacks']

# Totals
team_offense['Total Yards'] = team_offense['RushYards'] + team_offense['PassYards']
team_offense['Turnovers'] = team_offense['INTs'] + team_offense['Fumbles']

print(team_offense.sort_values(by='Total Yards', ascending=False).to_string())

In [ ]:
''' Team Advanced '''

ROUND = 3

team_advanced = pbp_data.loc[(pbp_data['Offensive Snap']) & (~pbp_data['Is Special Teams Play']), :].groupby(['posteam']).aggregate(
    Plays=('posteam', 'size'),
    PassPlays=('pass', 'sum'),
    RushPlays=('rush', 'sum'),
    EPA=('epa', 'sum'),
    RushEPA=('epa', lambda x: x[pbp_data['rush'] == 1].sum()),
    PassEPA=('epa', lambda x: x[pbp_data['pass'] == 1].sum()),
    Successes=('success', 'sum'),
    RushSuccesses=('success', lambda x: x[pbp_data['rush'] == 1].sum()),
    PassSuccesses=('success', lambda x: x[pbp_data['pass'] == 1].sum()),
)

team_advanced['EPA / Play'] = round(team_advanced['EPA'] / team_advanced['Plays'], ROUND)
team_advanced['Rush EPA / Play'] = round(team_advanced['RushEPA'] / team_advanced['RushPlays'], ROUND)
team_advanced['Pass EPA / Play'] = round(team_advanced['PassEPA'] / team_advanced['PassPlays'], ROUND)
team_advanced['Success Rate'] = round(team_advanced['Successes'] / team_advanced['Plays'], ROUND)
team_advanced['Rush Success Rate'] = round(team_advanced['RushSuccesses'] / team_advanced['RushPlays'], ROUND)
team_advanced['Pass Success Rate'] = round(team_advanced['PassSuccesses'] / team_advanced['PassPlays'], ROUND)

print(team_advanced.sort_values(by='EPA / Play', ascending=False).to_string())
# print(team_advanced.sort_values(by='EPA / Play', ascending=False)[['EPA / Play', 'Success Rate', 'Pass EPA / Play', 'Pass Success Rate', 'Rush EPA / Play', 'Rush Success Rate']].to_string())

In [ ]:
offense = get_team_offense_data(pbp_data)

print(offense.sort_values(by='EPA / Play', ascending=False).to_string())